1. Load the MS-COCO Dataset
To work with the MS-COCO dataset, the pycocotools library is useful for accessing image annotations. To simplify things here, we’ll assume the dataset is already downloaded and organized into folders for training and testing. Using tensorflow_datasets also provides an easy way to load a subset of MS-COCO directly.

In [2]:
!pip install tensorflow-datasets



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/5.3 MB ? eta -:--:--
                                              0.0/5.3 MB 1.3 MB/s eta 0:00:05
                                              0.0/5.3 MB 1.3 MB/s eta 0:00:05
                                              0.1/5.3 MB 1.3 MB/s eta 0:00:04
     -                                        0.2/5.3 MB 1.7 MB/s eta 0:00:03
     --                                       0.3/5.3 MB 1.7 MB/s eta 0:00:03
     --                                       0.3/5.3 MB 1.7 MB/s eta 0:00:03
     --                                       0.3/5.3 MB 1.7 MB/s eta 0:00:03
     --                                       0.3/5.3 MB 1.7 MB/s eta 0:00:03
     ---                                      0.4/5.3 MB 1.4 MB/s eta 0:00:04
     ---                                      0.4/5.3 MB 1.4 MB/s eta 0:00:04
     ---                                      0.4/5.3 MB 1.4 MB/s eta 0:00:04
     -----                                    0.7/5.3 MB 1.6 MB/s eta 0

In [ ]:
import tensorflow_datasets as tfds

# Load COCO dataset using TensorFlow Datasets
train_ds, test_ds = tfds.load('coco/2017', split=['train', 'validation'], shuffle_files=True)


c:\Users\visha\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  20%|██        | 1/5 [00:05<00:20,  5.13s/ url]

Dl Completed...:  20%|██        | 1/5 [12:08<00:20,  5.13s/ url]


2. Show the Number of Training and Testing Images

In [ ]:
# Count the number of images
train_count = tf.data.experimental.cardinality(train_ds).numpy()
test_count = tf.data.experimental.cardinality(test_ds).numpy()
print(f"Number of training images: {train_count}")
print(f"Number of testing images: {test_count}")


3. Plot Some Images

In [ ]:
import matplotlib.pyplot as plt

def plot_images(dataset, num_images=5):
    plt.figure(figsize=(10, 10))
    for i, sample in enumerate(dataset.take(num_images)):
        image = sample['image']
        plt.subplot(1, num_images, i + 1)
        plt.imshow(image)
        plt.axis("off")
    plt.show()

# Plot some training images
plot_images(train_ds)


4. Image Augmentation (Contrast, Flipping, and Rotation)

In [ ]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomContrast
from tensorflow.keras import Sequential

# Define the augmentation layers
augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.2),
    RandomContrast(0.2)
])

# Apply augmentation to the training dataset
train_ds = train_ds.map(lambda x: {'image': augmentation(x['image']), 'label': x['label']})


5. Show the Number of Training and Testing Images After Augmentation
Since augmentation is applied dynamically here, the number of images remains the same
6. Normalize the Training Data

In [ ]:
# Normalize pixel values between 0 and 1
def normalize_data(sample):
    sample['image'] = tf.image.convert_image_dtype(sample['image'], tf.float32)
    return sample

train_ds = train_ds.map(normalize_data)
test_ds = test_ds.map(normalize_data)


7. Build a Custom CNN for Image Classification


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(None, None, 3)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(80, activation='softmax')  # 80 classes in MS-COCO
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
history_cnn = cnn_model.fit(train_ds.batch(32), validation_data=test_ds.batch(32), epochs=10)


8. Show Training and Testing Accuracy for CNN


In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy for CNN model
plt.plot(history_cnn.history['accuracy'], label='Training Accuracy')
plt.plot(history_cnn.history['val_accuracy'], label='Validation Accuracy')
plt.title("CNN Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


9. Build a Faster R-CNN Model for Object Detection


In [ ]:
import tensorflow_hub as hub

# Load pre-trained Faster R-CNN model
faster_rcnn = hub.load("https://tfhub.dev/tensorflow/faster_rcnn/openimages_v4/inception_resnet_v2/1")

# Run inference on test images
for sample in test_ds.take(5):
    image = tf.image.convert_image_dtype(sample['image'], tf.float32)
    result = faster_rcnn(image[tf.newaxis, ...])
    # Process results, e.g., display bounding boxes and labels
    print(result)


10. Show Training and Testing Accuracy for Faster R-CNN
In object detection tasks, metrics such as mean Average Precision (mAP) are more commonly used than accuracy.

Comparison Before and After Augmentation
With Augmentation: Augmented images improve generalization, often resulting in higher validation accuracy.
Without Augmentation: Models may overfit, achieving high training accuracy but lower validation accuracy.